In [ ]:
import qcodes
import numpy as np

# Establishing Communication w/ Equipment

In [ ]:
# Load in Station
station = qcodes.Station()
station.load_config_file('station_config.yml')
station.load_all_instruments()

# Initialization Routine (SET/SHT)

## 1. Global Turn On